In [1]:
# from utils import *
from Bio import SeqIO
import pandas as pd
import numpy as np
from pysradb.sraweb import SRAweb
from datetime import datetime

/home/tobamo/miniconda3/envs/tobamo-model/lib/python3.10/site-packages/pysradb/utils.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [21]:
selected_contigs = {rec.id.replace('=', '_'): rec.seq for rec in SeqIO.parse('/home/tobamo/analize/project-tobamo/analysis/data/contigs/contigs_non_cellular_filtered.fasta', 'fasta')}
contig_table = pd.read_excel('results/contig_metadata_table_20250820.xlsx')
gt = pd.read_excel('/home/tobamo/analize/project-tobamo/analysis/data/domain_sci_input/ground_truth_20250630.xlsx')

In [25]:
gt_new = gt[gt['contig_name'].isin(selected_contigs.keys())]

In [18]:
# Get contig IDs that are in selected_contigs but not in contig_table
existing_contig_ids = set(contig_table['contig_id'].tolist())
selected_contig_ids = set(selected_contigs.keys())
missing_contig_ids = selected_contig_ids - existing_contig_ids

print(f"Missing contigs: {len(missing_contig_ids)}")
print(f"Missing contig IDs: {list(missing_contig_ids)}")

Missing contigs: 21
Missing contig IDs: ['k141_81265_flag_1_multi_5.0000_len_963_SRR7288019', 'k141_1210_flag_1_multi_2.0000_len_787_SRR1784304', 'k141_20647_flag_1_multi_3.0000_len_1328_SRR7288019', 'k141_33254_flag_1_multi_3.9093_len_637_SRR1784304', 'k141_83_flag_1_multi_17.0000_len_6273_SRR7288019', 'k141_70364_flag_1_multi_3.0000_len_1499_SRR7288019', 'k141_37653_flag_1_multi_6.0000_len_808_SRR1784304', 'k141_112743_flag_1_multi_2.0000_len_612_SRR7288019', 'k141_9354_flag_1_multi_12.0000_len_2527_SRR7288019', 'k119_47794_flag_1_multi_143.0000_len_5109_SRR5087336', 'k141_154710_flag_1_multi_10.8361_len_1727_SRR7288019', 'k141_12174_flag_1_multi_7.0000_len_1106_SRR1784304', 'k141_34437_flag_1_multi_4.0000_len_688_SRR7288019', 'k141_144373_flag_1_multi_35.0000_len_5656_SRR7288019', 'k141_135785_flag_1_multi_5.0000_len_1201_SRR7288019', 'k141_25901_flag_1_multi_2.0000_len_669_SRR7288019', 'k141_71034_flag_1_multi_2.0000_len_752_SRR7288019', 'k141_83781_flag_1_multi_3.0000_len_1189_SRR

In [19]:
missing_contig_ids

{'k119_47794_flag_1_multi_143.0000_len_5109_SRR5087336',
 'k141_112743_flag_1_multi_2.0000_len_612_SRR7288019',
 'k141_1210_flag_1_multi_2.0000_len_787_SRR1784304',
 'k141_12174_flag_1_multi_7.0000_len_1106_SRR1784304',
 'k141_135785_flag_1_multi_5.0000_len_1201_SRR7288019',
 'k141_144373_flag_1_multi_35.0000_len_5656_SRR7288019',
 'k141_154710_flag_1_multi_10.8361_len_1727_SRR7288019',
 'k141_20647_flag_1_multi_3.0000_len_1328_SRR7288019',
 'k141_25901_flag_1_multi_2.0000_len_669_SRR7288019',
 'k141_28952_flag_1_multi_1.0000_len_646_SRR1784304',
 'k141_33254_flag_1_multi_3.9093_len_637_SRR1784304',
 'k141_33900_flag_1_multi_6.0000_len_2289_SRR7288019',
 'k141_34437_flag_1_multi_4.0000_len_688_SRR7288019',
 'k141_35394_flag_1_multi_3.0000_len_1067_SRR1784304',
 'k141_37653_flag_1_multi_6.0000_len_808_SRR1784304',
 'k141_70364_flag_1_multi_3.0000_len_1499_SRR7288019',
 'k141_71034_flag_1_multi_2.0000_len_752_SRR7288019',
 'k141_81265_flag_1_multi_5.0000_len_963_SRR7288019',
 'k141_83781

In [26]:
# Create DataFrame for missing contigs
if missing_contig_ids:
    missing_data = []
    for contig_id in missing_contig_ids:
        sequence = str(selected_contigs[contig_id])
        corresponding_srr = contig_id.split('_')[-1] if '_' in contig_id else None
        assembler = 'spades' if 'NODE' in contig_id else 'megahit'
        contig_length = len(sequence)
        
        missing_data.append({
            'contig_id': contig_id,
            'sequence': sequence,
            'corresponding_srr': corresponding_srr,
            'assembler': assembler,
            'contig_length': contig_length
        })
    
    # Create DataFrame for missing contigs
    missing_df = pd.DataFrame(missing_data)
    
    # Add missing contigs to contig_table
    contig_table_updated = pd.concat([contig_table, missing_df], ignore_index=True)
    
    print(f"Updated contig_table now has {len(contig_table_updated)} rows")
    
    # Update the contig_table variable
    contig_table = contig_table_updated
else:
    print("No missing contigs found")

Updated contig_table now has 513 rows


In [27]:
contig_table

,contig_id,sequence,corresponding_srr,assembler,cluster_membership,known_or_potentially_novel_tobamovirus,contig_length,orf1_complete,orf1_partial,orf1_length,...,year_collected,study_accession,study_title,organism_name,submitter,country,publication_link,source_sample_category,genbank_accession_number,collection_date
0,string,string,string,string,number,TRUE/FALSE,number,TRUE/FALSE,TRUE/FALSE,number,...,number,string,string,string,string,string,string,category,string,2014
1,NODE_3447_length_1836_cov_746.295340_DRR146894,TTTTTTTTCTTCTTGAGTGTATGTTAAATATTTGTCAAATCCTTTC...,DRR146894,spades,NaN,False,1836,NaN,NaN,NaN,...,NaN,DRP004413,Phylogeny of the spider mite sub-family Tetran...,Eotetranychus uncatus,NaN,Japan:Gumma,NaN,NaN,NaN,2012
2,NODE_3346_length_1986_cov_333.205593_DRR146906,AAAAAAAACTTGCGAAGAGATCAAGGCAAGATTCGCTGATAAGCAG...,DRR146906,spades,NaN,False,1986,NaN,NaN,NaN,...,NaN,DRP004413,Phylogeny of the spider mite sub-family Tetran...,Schizotetranychus lespedezae,NaN,Japan:Ibaraki,NaN,NaN,NaN,2012
3,NODE_4755_length_1547_cov_49.162869_DRR146906,ACCAACGGGAACTACAAACCTTATCGCGACAACAAAGTCCTTGAGG...,DRR146906,spades,NaN,False,1547,NaN,NaN,NaN,...,NaN,DRP004413,Phylogeny of the spider mite sub-family Tetran...,Schizotetranychus lespedezae,NaN,Japan:Ibaraki,NaN,NaN,NaN,NaN
4,NODE_7104_length_1501_cov_4.732365_ERR1356733,CTTGATGTGTTTAGCACCAAGTTTCGAAATCAGCTTCAAAGGGTCG...,ERR1356733,spades,NaN,True,1501,NaN,NaN,NaN,...,NaN,ERP014782,Studying the presence and selection of antibio...,metagenome,EAWAG,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,k141_71034_flag_1_multi_2.0000_len_752_SRR7288019,GTTCATGTACCAGACGCCAAAACGACTCACGATCGCTTGCACCCGT...,SRR7288019,megahit,NaN,NaN,752,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
509,k141_83781_flag_1_multi_3.0000_len_1189_SRR728...,TTATAGTACTTCTCAGACTCTTCCAAAGGTTTGGACAGATCACACG...,SRR7288019,megahit,NaN,NaN,1189,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510,k141_35394_flag_1_multi_3.0000_len_1067_SRR178...,AAAATGAGAAACTGGAATTGATACAAAGGGGTTTTAAAGCCAATAC...,SRR1784304,megahit,NaN,NaN,1067,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,k141_33900_flag_1_multi_6.0000_len_2289_SRR728...,GTGTTGAAGATCTTAGTGGAAATATTGCTGGTATTGCTGGCAAAAC...,SRR7288019,megahit,NaN,NaN,2289,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
extra_contig_ids = existing_contig_ids - selected_contig_ids

print(f"Contigs in contig_table but not in selected_contigs: {len(extra_contig_ids)}")
print(f"Extra contig IDs: {list(extra_contig_ids)}")

Contigs in contig_table but not in selected_contigs: 3
Extra contig IDs: ['string', 'NODE_227_length_6529_cov_31.456441_SRR6233765', 'NODE_211_length_6524_cov_63.996754_SRR5087405']


### OLD MAKING OF

todo: change paths to relative ones in project-tobamo

In [3]:
# import data
non_celular_path = '../data/contigs/contigs_non_cellular.fasta'
gt = pd.read_excel('../data/domain_sci_input/ground_truth_20250630.xlsx')
template_cols = pd.read_excel('../data/domain_sci_input/contig_report_columns.xlsx').columns
sra_metadata_template_cols = pd.read_excel('../data/domain_sci_input/mapping_sra_metadata.xlsx')

# # SRRs to remove (after manual curation on 4.7.2025 (WA correpondence)) due to replicated uploads of same studies (Panonychus citri (SRP187337, SRP188804) in pa VIZIONS (ERP108694, ERP006046))
# SRRs_to_remove = ['ERR2737479', 'SRR8749694', 'SRR8749695', 'SRR8749695', 'SRR8749693']

# Create an empty DataFrame with the template columns
df = pd.DataFrame(columns=template_cols)

# parse fasta
records = list(SeqIO.parse(non_celular_path, 'fasta'))

# strip headers
gt.columns = gt.columns.str.strip()

# filter records based on ground truth
contig_names = gt.contig_name.unique()
records = [r for r in records if r.id in contig_names]

In [6]:
# start filling df columns
df['contig_id'] = [rec.id for rec in records]
df['sequence'] = [str(rec.seq) for rec in records]
df['corresponding_srr'] = df['contig_id'].str.split('_').str[-1]

# # remove SRRs that are in the SRRs_to_remove list
# df = df.copy()
# df = df[~df['corresponding_srr'].isin(SRRs_to_remove)]

df['assembler'] = np.where(df['contig_id'].str.contains('NODE'), 'spades', 'megahit')
df['contig_length'] = [len(seq) for seq in df['sequence']]

# make mapper, add ground truth category
gt_category_mapper = gt.set_index('contig_name')['category'].to_dict()
df['ground_truth_category'] = df['contig_id'].map(gt_category_mapper)

df['known_or_potentially_novel_tobamovirus'] = np.where(df['ground_truth_category'].isin([1,2,8]), True, False)

import SRA metadata

In [96]:
## RUN ONLY ONCE
# # Get unique SRR values
# srr_list = df['Corresponding SRR'].unique().tolist()

# # Initialize SRAweb
# db = SRAweb()

# # Fetch metadata for all SRRs
# metadata = db.sra_metadata(srr_list, detailed=True)

# # save
# metadata.to_csv('results/sra_metadata.csv', index=False)

In [7]:
# import metadata
metadata = pd.read_csv('results/sra_metadata.csv')

select columns for parsing

In [8]:
# create sra_metadata_columnames_mapper
sra_mapper = sra_metadata_template_cols.dropna().set_index('run_accession')['corresponding_srr'].to_dict()

# keep selected columns
metadata_sub = metadata[['run_accession', 'study_accession', 'study_title', 'organism_name', 'collection_date', 'geo_loc_name', 'insdc center name']].drop_duplicates()
metadata_sub.rename(columns={'run_accession': 'corresponding_srr'}, inplace=True)

In [9]:
# Create mapping dictionaries for each metadata column
col_map = {
    'collection_date': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['collection_date'])),
    'study_accession': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['study_accession'])),
    'study_title': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['study_title'])),
    'organism_name': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['organism_name'])),
    'country': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['geo_loc_name'])), 
    'submitter': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['insdc center name'])),
}

# Map each metadata column to df using the corresponding_srr
for col, mapper in col_map.items():
    df[col] = df['corresponding_srr'].map(mapper)

In [ ]:
# date = datetime.now().strftime('%Y-%m-%d')
date = '2025-07-07'
df.to_csv(f'results/{date}_contigs_table.csv', index=False) # have to add some info manually